In [3]:
import pandas as pd

df = pd.read_csv("LungCancer.csv")




In [4]:
print("First 10 rows:")
display(df.head(10))
print("\nDataset Description:")
display(df.describe())

First 10 rows:


,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,65,1,1,1,2,2,1,2,2,2,2,2,2,1,NO
1,F,55,1,2,2,1,1,2,2,2,1,1,1,2,2,NO
2,F,78,2,2,1,1,1,2,1,2,1,1,2,1,1,YES
3,M,60,2,1,1,1,2,1,2,1,1,2,1,2,2,YES
4,F,80,1,1,2,1,1,2,1,2,1,1,1,1,2,NO
5,F,58,1,1,1,2,2,2,2,1,2,2,1,1,2,YES
6,F,70,1,1,1,2,2,1,2,2,2,2,2,2,1,YES
7,F,74,2,2,1,1,1,1,2,1,1,1,1,2,1,NO
8,M,77,1,2,1,2,1,1,1,1,2,1,1,1,2,NO
9,F,67,2,2,2,2,1,2,2,1,2,1,2,1,1,NO



Dataset Description:


,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,55.169000,1.491000,1.514000,1.494000,1.499000,1.509667,1.489667,1.506667,1.497333,1.491333,1.510667,1.488000,1.489667,1.498667
std,14.723746,0.500002,0.499887,0.500047,0.500082,0.499990,0.499977,0.500039,0.500076,0.500008,0.499970,0.499939,0.499977,0.500082
min,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,42.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,55.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000
75%,68.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,80.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000


In [5]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
print("\nNumber of duplicates:", df.duplicated().sum())
print(df.head())


Number of duplicates: 0
  GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0      M   65        1               1        1              2   
1      F   55        1               2        2              1   
2      F   78        2               2        1              1   
3      M   60        2               1        1              1   
4      F   80        1               1        2              1   

   CHRONIC_DISEASE  FATIGUE  ALLERGY  WHEEZING  ALCOHOL_CONSUMING  COUGHING  \
0                2        1        2         2                  2         2   
1                1        2        2         2                  1         1   
2                1        2        1         2                  1         1   
3                2        1        2         1                  1         2   
4                1        2        1         2                  1         1   

   SHORTNESS_OF_BREATH  SWALLOWING_DIFFICULTY  CHEST_PAIN LUNG_CANCER  
0                    2         

In [6]:
from sklearn.preprocessing import LabelEncoder
columns_to_drop=['ANXIETY','PEER_PRESSURE','CHRONIC_DISEASE','GENDER','AGE','ALCOHOL_CONSUMING']
df=df.drop(columns=columns_to_drop)
label_encoder = LabelEncoder()
df['LUNG_CANCER'] = label_encoder.fit_transform(df['LUNG_CANCER'])
columns_to_transform = ['SMOKING', 'YELLOW_FINGERS', 'FATIGUE', 'ALLERGY', 'WHEEZING', 'COUGHING','SHORTNESS_OF_BREATH','SWALLOWING_DIFFICULTY','CHEST_PAIN']
df[columns_to_transform] = df[columns_to_transform].replace({1: 0, 2: 1})
df.head()

,SMOKING,YELLOW_FINGERS,FATIGUE,ALLERGY,WHEEZING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,0,0,0,1,1,1,1,1,0,0
1,0,1,1,1,1,0,0,1,1,0
2,1,1,1,0,1,0,1,0,0,1
3,1,0,0,1,0,1,0,1,1,1
4,0,0,1,0,1,0,0,0,1,0


In [7]:
y = df['LUNG_CANCER']
X = df.drop(columns=['LUNG_CANCER'])

feature_column_names = X.columns.tolist()
print("Feature column names (X):")
print(feature_column_names)
print("\nTarget (y):")
print(y.head())

Feature column names (X):
['SMOKING', 'YELLOW_FINGERS', 'FATIGUE', 'ALLERGY', 'WHEEZING', 'COUGHING', 'SHORTNESS_OF_BREATH', 'SWALLOWING_DIFFICULTY', 'CHEST_PAIN']

Target (y):
0    0
1    0
2    1
3    1
4    0
Name: LUNG_CANCER, dtype: int32


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=500, n_features=9, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

KNNmodel = KNeighborsClassifier(n_neighbors=10)
KNNmodel.fit(X_train_scaled, y_train)

y_pred = KNNmodel.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))



Accuracy: 0.83


In [10]:
from sklearn.linear_model import LogisticRegression
LGmodel = LogisticRegression()
LGmodel.fit(X_train_scaled, y_train)

y_pred = LGmodel.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f" {accuracy }")


 0.87


In [11]:
from sklearn.naive_bayes import GaussianNB
NBmodel = GaussianNB()
NBmodel.fit(X_train, y_train)

y_pred = NBmodel.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f" Accuracy: {accuracy }")


 Accuracy: 0.86


In [12]:


from xgboost import XGBClassifier


XGBmodel = XGBClassifier(eval_metric='mlogloss') 
XGBmodel.fit(X_train, y_train)

y_pred = XGBmodel.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f" Accuracy: {accuracy }")


 Accuracy: 0.93


In [13]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(XGBmodel, X_train_scaled, y_train, cv=5)
print("CV Accuracy:", scores.mean())


CV Accuracy: 0.9349999999999999


In [14]:
import numpy as np
my_record = np.array([0, 0, 0, 1, 1, 1, 1, 1, 0]).reshape(1, -1)
	

# 5. Predict the class
prediction = XGBmodel.predict(my_record)
probability = XGBmodel.predict_proba(my_record)

print("Your Record:", my_record)
print("Predicted Class:", prediction[0])
print("Prediction Probabilities:", probability[0])

Your Record: [[0 0 0 1 1 1 1 1 0]]
Predicted Class: 0
Prediction Probabilities: [0.9722024  0.02779757]


In [15]:
pip install xgboost


Note: you may need to restart the kernel to use updated packages.


In [16]:

import joblib

joblib.dump(XGBmodel, 'lung_cancer_model.pkl')

print("Model saved as lung_cancer_model.pkl")

Model saved as lung_cancer_model.pkl
